# Classification Report with GLCM and BERT

## Create and Save CSV

In [ ]:
# Import library 
import numpy as np
import pandas as pd
import os
from PIL import Image

import tensorflow as tf

from keras.applications import ResNet50

from skimage.feature import greycomatrix, greycoprops

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import urllib.request
from gensim.utils import simple_preprocess
import os, re

from sklearn.metrics.pairwise import cosine_similarity

from scipy.spatial.distance import cosine

# lib's
from pickle import dump , load

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

import keras
from keras.models import Model, load_model
from keras.layers import Input, Dense, Concatenate, Flatten
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import h5py

import glob


In [ ]:
# function
# Function to extract visual features using ResNet50
def extract_visual_features(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize image for ResNet50 input
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)

    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    visual_features = model.predict(img_array, verbose = 0)

    return visual_features

# Function to compute GLCM texture features
def compute_texture_features(image_path):
    # Define your GLCM properties (distances and angles)
    distances = [1, 3, 5, 3, 1, 3, 5]
    angles = [0, 0, 0, np.pi/4, np.pi/2, np.pi/2, np.pi/2]

    glcm_features = []
    for distance, angle in zip(distances, angles):
        img_gray = Image.open(image_path).convert('L')  # Convert image to grayscale
        img_gray_array = np.array(img_gray)

        # Compute GLCM properties
        glcm = greycomatrix(img_gray_array, distances=[distance], angles=[angle], levels=256, symmetric=True, normed=True)
        contrast = greycoprops(glcm, 'contrast')[0, 0]
        dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
        homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
        energy = greycoprops(glcm, 'energy')[0, 0]
        correlation = greycoprops(glcm, 'correlation')[0, 0]



        glcm_features.extend([contrast, dissimilarity, homogeneity, energy, correlation])

    return np.array(glcm_features) # np.array([contrast, dissimilarity, homogeneity, energy, correlation])

# Load image data and compute features
def load_data_and_compute_features(data_dir):
    visual_features = []
    texture_features = []
    labels = []

    classes = os.listdir(data_dir)
    class_to_index = {cls: idx for idx, cls in enumerate(classes)}
    # print(f"classes : {classes} \nclass_to_index : {class_to_index}")

    for cls in tqdm(classes, desc="Processing images"):
        cls_dir = os.path.join(data_dir, cls)

        for img_name in os.listdir(cls_dir):
            img_path = os.path.join(cls_dir, img_name)

            # Extract visual features using ResNet50
            visual_feat = extract_visual_features(img_path)
            visual_features.append(visual_feat)
            # print(f"visual_features : {visual_features}")

            # Compute texture features using GLCM
            texture_feat = compute_texture_features(img_path)
            texture_features.append(texture_feat)
            # print(f"texture_features : {texture_features}")

            # Assign label index
            labels.append(class_to_index[cls])
            # print(f"img_path : {img_path} || labels : {class_to_index[cls]} \n")
            # print(f"texture_features : {texture_feat} \n")

    visual_features = np.array(visual_features)
    texture_features = np.array(texture_features)
    labels = np.array(labels)

    return visual_features, texture_features, labels

In [ ]:
# load_data_and_compute_features
data_directory = 'dataset path'
visual_features, texture_features, labels = load_data_and_compute_features(data_directory)
print('Done')
print(visual_features.shape, texture_features.shape, labels.shape)

In [ ]:
data_vt = pd.DataFrame(np.concatenate([visual_features.reshape(visual_features.shape[0], -1), texture_features], axis=1))
data_vt['labels'] = labels

data_vt.to_csv('dataset_features.csv', index=False)

# ---------------------------------------------------------------------------------------------

## Model Train and Test

In [ ]:
# Variable's
min_max_scaler_file = 'min_max_scaler.pkl'

In [ ]:
# Function
# Build and train a classification model
def build_and_train_model(visual_features, texture_features, labels):
    # Concatenate visual and texture features
    combined_features = np.concatenate([visual_features,
                                        texture_features],
                                       axis=1)


    # Normalize the features
    min_max_scaler = StandardScaler()
    train_normalized_texture_features = min_max_scaler.fit_transform(combined_features)
    train_normalized_texture_features_df = pd.DataFrame(train_normalized_texture_features)

    # save
    dump(min_max_scaler, open(min_max_scaler_file, 'wb'))

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(train_normalized_texture_features_df,
                                                        labels,
                                                        test_size=0.3,
                                                        random_state=42,
                                                        stratify=labels)

    # Define classification model
    input_layer = Input(shape=(combined_features.shape[1],))
    hidden_layer = Dense(128, activation='relu')(input_layer)
    output_layer = Dense(len(np.unique(labels)), activation='softmax')(hidden_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.summary()
    model.compile(optimizer=Adam(lr=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train,
                        epochs=100,
                        batch_size=64,
                        validation_data=(X_test, y_test))

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc)+1)

    plt.plot(epochs, acc, label='training accuracy')
    plt.plot(epochs, val_acc, label='validation accuracy')
    plt.legend()
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.title('Training validation Accuracy')
    plt.show()

    plt.plot(epochs, loss, label='training loss')
    plt.plot(epochs, val_loss, label='validation loss')
    plt.legend()
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.title('Training validation Loss')
    plt.show()

    return model # history # model

In [ ]:
# load csv
data_vtl = pd.read_csv('dataset_features.csv')
visual_features, texture_features, labels = data_vtl.iloc[:,:2048], data_vtl.iloc[:,2048:-1], data_vtl.iloc[:,-1]

In [ ]:
# Build and train the classification model
classification_model = build_and_train_model(visual_features, texture_features, labels)
classification_model

In [ ]:
# save model and architecture to single file
classification_model.save("OSFE_model.h5")
print("Saved model to disk")

Saved model to disk


# -------------------------------------------------------------------------------------

## Classification Report

In [ ]:
# Load models
classification_model = load_model('OSFE_model.h5')
min_max_scaler_model = load(open('min_max_scaler.pkl', 'rb'))
class_labels = ['add class labels of training and zero-shot classes']

### CR without BERT

In [ ]:
image_path, image_label, pred_image_label, act_label, pred_label = [], [], [], [], []

for img_path in glob.glob(r"test_datapath/*/*"):
#     print(f"img_path : {img_path}")
    image_path.append(img_path)
    image_label.append(img_path.split('\\')[-2])
    act_label.append(class_labels.index(img_path.split('\\')[-2]))

    # Extract visual features using ResNet50
    visual_features = extract_visual_features(img_path)
#     print(f"visual_features : {visual_features.shape} || {type(visual_features)}")

    # Compute texture features using GLCM
    texture_features = compute_texture_features(img_path)
#     print(f"texture_feat : {texture_features.shape} || {type(texture_features)}")

    # Concatenate visual and texture features
    l1 = visual_features[0].tolist()
    l2 = texture_features.tolist()
    l1.extend(l2)
    combined_features = np.array(l1)
#     print(f"combined_features : {combined_features}")

    # Normalize the features
    train_normalized_texture_features = min_max_scaler_model.transform([combined_features])
    train_normalized_texture_features_df = pd.DataFrame(train_normalized_texture_features)
#     print(f"train_normalized_texture_features_df : {train_normalized_texture_features_df}")

#     model.predict
    predicted_value = classification_model.predict(train_normalized_texture_features_df, verbose = 0)
#     print(f"predicted_value : {np.argmax(predicted_value)} || {class_labels[np.argmax(predicted_value)]}")
    pred_image_label.append(class_labels[np.argmax(predicted_value)])
    pred_label.append(np.argmax(predicted_value))



### CR with BERT

#Classification with NLP Autoencoder#


In [ ]:
from tensorflow.keras.models import load_model
import os

# Define the model folder path
model_folder_path = 'add model folder path'

# Load the encoder model
encoder_model_path = os.path.join(model_folder_path, 'add path of encoder model file')
encoder = load_model(encoder_model_path)

# Load the autoencoder model
autoencoder_model_path = os.path.join(model_folder_path, 'add path of autoencoder model file')
autoencoder = load_model(autoencoder_model_path)

print("Models loaded successfully!")

In [7]:
def find_similar_sentences(target_sentence_index, sentence_to_latent, top_n=10):
    target_latent = sentence_to_latent[target_sentence_index]
    similarities = []
    for index, latent in sentence_to_latent.items():
        if index != target_sentence_index:
            similarity = 1 - cosine(target_latent, latent)
            similarities.append((index, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

def predict_with_autoencoder(input_word_index, sentence_to_latent, top_n=5):
    # Find sentences similar to the first sentence
    similar_sentences = find_similar_sentences(input_word_index, sentence_to_latent, top_n=top_n)
    similar_words = [custom_sentences[index] for index, score in similar_sentences]
    return similar_words

In [ ]:
# Example custom dataset
custom_sentences = ['add labels of training and zero shot classes']

# animals_list_file_path = os.path.join('..', 'Dataset', 'artefact1', 'animals.txt')
# with open(animals_list_file_path, "r") as f:
#custom_sentences = [line[:-1] for line in f]

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

embeddings_index = load_glove_embeddings(r"D:\ZSL\CNN\glove.6B\glove.6B.300d.txt")  # Using 50d GloVe embeddings
print(f"embeddings_index : {type(embeddings_index)} \n")

# Convert sentences to embeddings
def sentence_to_embedding(sentence, embeddings_index):
    words = simple_preprocess(sentence)
    valid_words = [embeddings_index[word] for word in words if word in embeddings_index]
    if valid_words:
        return np.mean(valid_words, axis=0)
    else:
        return np.zeros(300) # Using 50d GloVe embeddings

sentence_embeddings = np.array([sentence_to_embedding(sentence, embeddings_index) for sentence in custom_sentences])
print(f"sentence_embeddings : {type(sentence_embeddings)} \n")

In [ ]:
# Encode sentences to get their latent representations
sentence_latents = encoder.predict(sentence_embeddings)
# print(f"sentence_latents : {sentence_latents}")

# Create a dictionary to map sentences to their latent representations
sentence_to_latent = {i: sentence_latents[i] for i in range(len(custom_sentences))}
print(f"sentence_to_latent : {sentence_to_latent}")

In [ ]:

j=0
counter_rat=0
counter_pig=0
counter_raccoon=0
counter_cat=0
counter_leopard=0
counter_humpback=0
counter_hippopotamus=0
counter_panda=0
counter_chimp=0
counter_seal=0

Autoencoder_counter=0


# Load models
classification_model = load_model('OSFE_model.h5')
min_max_scaler_model = load(open('min_max_scaler.pkl', 'rb'))
class_labels = ['add class labels of training and zero-shot classes']



image_path, image_label, pred_image_label, act_label, pred_label, similar_words = [], [], [], [], [], []


# for img_path in glob.glob(folder_path):
for img_path in glob.glob(r"Path of test classes\*\*"):
    
    
    print(f"img_path : {img_path}")
    image_path.append(img_path)
    image_label.append(img_path.split('\\')[-2])
    act_label.append(class_labels.index(img_path.split('\\')[-2]))

    # Extract visual features using ResNet50
    visual_features = extract_visual_features(img_path)


    # Compute texture features using GLCM
    texture_features = compute_texture_features(img_path)


    # Concatenate visual and texture features
    l1 = visual_features[0].tolist()
    l2 = texture_features.tolist()
    l1.extend(l2)
    combined_features = np.array(l1)


    # Normalize the features
    train_normalized_texture_features = min_max_scaler_model.transform([combined_features])
    train_normalized_texture_features_df = pd.DataFrame(train_normalized_texture_features)


#     model.predict
    predicted_value = classification_model.predict(train_normalized_texture_features_df)
    print(f"predicted_value : {np.argmax(predicted_value)} || {class_labels[np.argmax(predicted_value)]}")
    pred_image_label.append(class_labels[np.argmax(predicted_value)])
    pred_label.append(np.argmax(predicted_value))

    # Get the predicted label
    predicted_class_label = class_labels[np.argmax(predicted_value)]
    # print(predicted_class_label)

  
    if predicted_class_label in custom_sentences:
        similar_words_autoencoder = predict_with_autoencoder(custom_sentences.index(predicted_class_label), sentence_to_latent, top_n=5)
    else:
        print(f"Label {predicted_class_label} not found in custom_sentences.")



    similar_words_autoencoder = predict_with_autoencoder(custom_sentences.index(predicted_class_label), sentence_to_latent, top_n=10) # class_labels
    print(f"similar_words_autoencoder : {similar_words_autoencoder}")


    for i in similar_words_autoencoder:
                 if (img_path.split("\\")[-2]) in i:  
                      Autoencoder_counter += 1
                      print(img_path.split("\\")[-2])

    
